In [2]:
%run 02_tool_use.ipynb

Dates        : ['2026-02-25', '2026-02-26', '2026-02-27', '2026-02-28', '2026-03-01', '2026-03-02', '2026-03-03']
Temp max (°C): [10.5, 9.5, 9.5, 7.5, 5.8, 11.4, 11.3]
Temp min (°C): [4.4, 3.3, 2.3, 1.7, 0.3, 1.6, 4.9]
Précip (mm)  : [0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0]
Neige (cm)   : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Vent (km/h)  : [10.3, 10.4, 10.7, 6.5, 7.9, 11.7, 10.0]
Utilisation de l'outil : get_weather avec {'latitude': 44.5167, 'longitude': 6.5333, 'station_name': 'Les Orres'}
Voici les prévisions d'enneigement pour **Les Orres** cette semaine (du 25 février au 3 mars 2026) :

---

### ❄️ Résumé de l'enneigement – Les Orres (alt. ~1 310 m)

| Jour | Temp. Min / Max | Chutes de neige | Hauteur de neige estimée |
|------|----------------|-----------------|--------------------------|
| 📅 Mer. 25 fév. | 6°C / 13°C | Aucune | ~32 cm |
| 📅 Jeu. 26 fév. | 5°C / 12°C | Aucune | ~28 cm |
| 📅 Ven. 27 fév. | 3°C / 11°C | Aucune | ~24 cm |
| 📅 Sam. 28 fév. | 1°C / 9°C | Aucune | ~17 cm |


In [3]:
class LLMSnowAgent:
    def __init__(self):
        self.client = anthropic.Anthropic()
        self.conversation_history = []
        self.system_prompt = """Tu es LLMSnow, un assistant spécialisé dans les prévisions météorologiques pour les stations de ski. Tu fournis des informations sur les conditions de neige, les températures, et les prévisions à court terme pour aider les skieurs à planifier leurs journées sur les pistes.

Ton rôle:
- faire un onboard: comprendre le profil de l'utilisateur (niveau de ski, préférences de stations, avec enfants ?, dates, etc.)
- Récupérer les conditions météo réelles via tes outils
- Générer une packing list personnalisée selon la météo et le profil
- Proposer des activités adaptées aux conditions

Commence toujours par collecter les informations essentielles avant de faire des recommandations.
Sois précis, pratique et enthousiaste — tu parles à quelqu'un qui prépare ses vacances !"""

    def chat(self, user_message: str) -> str:
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        messages = self.conversation_history.copy()
        
        while True:
            response = self.client.messages.create(
                model="claude-sonnet-4-6",
                max_tokens=4096,
                system=self.system_prompt,
                messages=messages,
                tools=[weather_tool]
            )
            
            if response.stop_reason == "end_turn":
                assistant_message = response.content[0].text
                self.conversation_history.append({
                    "role": "assistant",
                    "content": assistant_message
                })
                return assistant_message
            
            if response.stop_reason == "tool_use":
                messages.append({
                    "role": "assistant",
                    "content": response.content
                })
                tool_results = []
                for block in response.content:
                    if block.type == "tool_use":
                        result = execute_tool(block.name, block.input)
                        tool_results.append({
                            "type": "tool_result",
                            "tool_use_id": block.id,
                            "content": result
                        })
                messages.append({
                    "role": "user",
                    "content": tool_results
                })
                
# Exemple d'utilisation
agent = LLMSnowAgent()
user_input = "Salut LLMSnow ! Je prépare mes vacances de ski aux Orres du 27 février au 7 mars. J'aimerais savoir quelles sont les conditions de neige prévues pour cette période, et si tu peux me faire une packing list personnalisée en fonction de la météo ?"
print("---")
print(agent.chat(user_input))
print("---")
print(agent.chat("Nous sommes 2 adultes, en snow, niveau débutant à intermédiaire. Nous avons nos propres boards et chaussures. Nous avons une voiture pour transporter nos affaires. Nous ne voulons pas skier tous les jours, on aimerait aussi faire des activités hors ski. Nous rejoignons la famille sur place: 4 adultes"))

---
J'ai les données météo, mais elles ne couvrent que jusqu'au 3 mars. Pendant ce temps, dis-moi en plus sur toi pour que je personnalise tout ! 🙌 Voici déjà un premier aperçu des conditions :

---

## 🏔️ Conditions météo aux Orres — 27 fév au 3 mars

> ⚠️ **Note importante :** Les données ci-dessous sont mesurées à **1 292 m d'altitude** (station météo en vallée). Les pistes des Orres montent jusqu'à **2 720 m** — la neige y est systématiquement bien meilleure en altitude !

### ❄️ Enneigement actuel (en bas de station)
| Date | Hauteur de neige (bas) | Neige fraîche | Vent max |
|------|----------------------|---------------|----------|
| **27 fév** | ~24 cm | ❌ 0 cm | 11,5 km/h 🟢 |
| **28 fév** | ~17 cm | ❌ 0 cm | 8,4 km/h 🟢 |
| **1 mar** | ~13 cm | ❌ 0 cm | 6,9 km/h 🟢 |
| **2 mar** | ~10 cm | ❌ 0 cm | 11,9 km/h 🟢 |
| **3 mar** | ~3 cm | ❌ 0 cm | 10,0 km/h 🟢 |

### 🌡️ Températures
| Date | Min | Max |
|------|-----|-----|
| **27 fév** | +3°C | +11°C |
| **28 fév** | +1°C | +9°C |
|